In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping


In [3]:
df= pd.read_csv('LSTM-data.csv')
df.head()

,Unnamed: 0,날짜,kospi200,kosdaq,dow,nikkei,ssec,usd_krw,cny_krw,jpy_krw,콜금리,국고채,wti,brent,gold,vix,Target
0,1738,2017-04-17,278.10,629.47,20636.92,18355.26,3222.17,1133.02,164.63,10.4037,1.23,2.165,52.73,55.36,1291.9,14.66,1
1,1737,2017-04-18,278.23,634.73,20636.92,18418.59,3196.71,1133.02,166.03,10.5386,1.23,2.165,52.73,55.36,1291.9,14.66,0
2,1736,2017-04-19,276.49,635.99,20523.28,18432.20,3170.69,1142.64,166.05,10.5056,1.23,2.165,52.33,54.89,1294.1,14.42,1
3,1735,2017-04-20,277.76,635.80,20404.49,18430.49,3172.10,1143.69,165.37,10.4108,1.24,2.165,50.62,52.93,1283.4,14.93,1
4,1734,2017-04-21,280.05,634.96,20578.71,18620.75,3173.15,1138.12,164.77,10.4008,1.24,2.165,50.27,52.99,1283.8,14.15,1


In [4]:
df_fi=df.drop(['Unnamed: 0','usd_krw', 'cny_krw','gold','wti','jpy_krw','vix','국고채'], axis=1)

df_fi.head()

,날짜,kospi200,kosdaq,dow,nikkei,ssec,콜금리,brent,Target
0,2017-04-17,278.10,629.47,20636.92,18355.26,3222.17,1.23,55.36,1
1,2017-04-18,278.23,634.73,20636.92,18418.59,3196.71,1.23,55.36,0
2,2017-04-19,276.49,635.99,20523.28,18432.20,3170.69,1.23,54.89,1
3,2017-04-20,277.76,635.80,20404.49,18430.49,3172.10,1.24,52.93,1
4,2017-04-21,280.05,634.96,20578.71,18620.75,3173.15,1.24,52.99,1


In [5]:
# 데이터를 역순으로 정렬하여 전체 데이터의 70% 학습, 30% 테스트에 사용
df_fi = df_fi[::-1]
df_fi.head()

,날짜,kospi200,kosdaq,dow,nikkei,ssec,콜금리,brent,Target
1738,2024-05-14,371.04,862.22,39431.64,38314.50,3146.29,3.581,83.36,0
1737,2024-05-13,370.98,854.43,39431.64,38179.46,3148.17,3.581,83.36,1
1736,2024-05-10,371.08,864.16,39387.76,38257.00,3155.65,3.581,83.88,0
1735,2024-05-09,368.84,870.15,39055.73,38070.50,3155.43,3.581,83.58,1
1734,2024-05-08,374.09,872.47,38883.94,38202.37,3128.48,3.581,82.99,0


In [6]:
# 패키지 임포트
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score, roc_curve
from sklearn.preprocessing import MinMaxScaler

# 날짜 칼럼을 datetime 객체로 변환
df_fi['날짜'] = pd.to_datetime(df_fi['날짜'])
df_fi.set_index('날짜', inplace=True)

# 타깃 변수 만들어주기
# 단순히 다음 날 오르면 1, 하락하면 0
df_fi['Target'] = (df_fi['kospi200'].shift(-1) > df_fi['kospi200']).astype(int)


In [7]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np

# 피처와 타겟을 분리합니다
features = df_fi.drop('Target', axis=1)
target = df_fi['Target']

# 피처 스케일링
scaler = MinMaxScaler()
scaled_features = scaler.fit_transform(features)

# LSTM 입력 형식으로 변환 (samples, time steps, features)
time_steps = 5  # 이전 5일의 데이터를 사용하여 예측

X = []
y = []

for i in range(time_steps, len(scaled_features)):
    X.append(scaled_features[i-time_steps:i])
    y.append(target.values[i])

X, y = np.array(X), np.array(y)


In [8]:
split_ratio = 0.7  # 훈련 세트 비율
split_index = int(split_ratio * len(X))

X_train, y_train = X[:split_index], y[:split_index]
X_test, y_test = X[split_index:], y[split_index:]

In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

model = Sequential()
model.add(LSTM(units=128, return_sequences=True, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.1))
model.add(LSTM(units=64))
model.add(Dropout(0.1))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
earlystop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)


Epoch 1/100
31/31 [==============================] - 6s 52ms/step - loss: 0.6968 - accuracy: 0.4907 - val_loss: 0.6954 - val_accuracy: 0.4486
Epoch 2/100
31/31 [==============================] - 1s 24ms/step - loss: 0.6917 - accuracy: 0.5247 - val_loss: 0.6876 - val_accuracy: 0.5514
Epoch 3/100
31/31 [==============================] - 1s 27ms/step - loss: 0.6926 - accuracy: 0.5165 - val_loss: 0.6896 - val_accuracy: 0.5514
Epoch 4/100
31/31 [==============================] - 1s 32ms/step - loss: 0.6916 - accuracy: 0.5216 - val_loss: 0.6878 - val_accuracy: 0.5514
Epoch 5/100
31/31 [==============================] - 1s 25ms/step - loss: 0.6922 - accuracy: 0.5113 - val_loss: 0.6890 - val_accuracy: 0.5514
Epoch 6/100
31/31 [==============================] - 1s 18ms/step - loss: 0.6920 - accuracy: 0.5155 - val_loss: 0.6910 - val_accuracy: 0.5432
Epoch 7/100
31/31 [==============================] - 1s 19ms/step - loss: 0.6920 - accuracy: 0.5206 - val_loss: 0.6902 - val_accuracy: 0.5514
Epoch 

In [10]:
model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2, callbacks=[earlystop])

Epoch 1/100
31/31 [==============================] - 1s 19ms/step - loss: 0.6914 - accuracy: 0.5258 - val_loss: 0.6886 - val_accuracy: 0.5514
Epoch 2/100
31/31 [==============================] - 0s 16ms/step - loss: 0.6882 - accuracy: 0.5227 - val_loss: 0.6897 - val_accuracy: 0.5597
Epoch 3/100
31/31 [==============================] - 1s 17ms/step - loss: 0.6881 - accuracy: 0.5186 - val_loss: 0.6892 - val_accuracy: 0.5514
Epoch 4/100
31/31 [==============================] - 1s 17ms/step - loss: 0.6888 - accuracy: 0.5351 - val_loss: 0.6919 - val_accuracy: 0.5144
Epoch 5/100
31/31 [==============================] - 1s 16ms/step - loss: 0.6875 - accuracy: 0.5206 - val_loss: 0.6888 - val_accuracy: 0.5514
Epoch 6/100
31/31 [==============================] - 0s 15ms/step - loss: 0.6892 - accuracy: 0.5103 - val_loss: 0.6903 - val_accuracy: 0.5638
Epoch 7/100
31/31 [==============================] - 0s 15ms/step - loss: 0.6860 - accuracy: 0.5340 - val_loss: 0.6908 - val_accuracy: 0.5514
Epoch 

In [11]:
predictions = model.predict(X_test)
predictions = (predictions > 0.5).astype(int)  # 이진 분류 문제이므로 0.5를 기준으로 예측


17/17 [==============================] - 1s 5ms/step


In [12]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_test, predictions)
print(f"Accuracy: {accuracy}")


Accuracy: 0.5508637236084453


----------------------------------------------------

In [13]:
df_att=df.drop(['Unnamed: 0','국고채'], axis=1)

df_att.head()

,날짜,kospi200,kosdaq,dow,nikkei,ssec,usd_krw,cny_krw,jpy_krw,콜금리,wti,brent,gold,vix,Target
0,2017-04-17,278.10,629.47,20636.92,18355.26,3222.17,1133.02,164.63,10.4037,1.23,52.73,55.36,1291.9,14.66,1
1,2017-04-18,278.23,634.73,20636.92,18418.59,3196.71,1133.02,166.03,10.5386,1.23,52.73,55.36,1291.9,14.66,0
2,2017-04-19,276.49,635.99,20523.28,18432.20,3170.69,1142.64,166.05,10.5056,1.23,52.33,54.89,1294.1,14.42,1
3,2017-04-20,277.76,635.80,20404.49,18430.49,3172.10,1143.69,165.37,10.4108,1.24,50.62,52.93,1283.4,14.93,1
4,2017-04-21,280.05,634.96,20578.71,18620.75,3173.15,1138.12,164.77,10.4008,1.24,50.27,52.99,1283.8,14.15,1


In [14]:
# 피쳐 중요도 값을 딕셔너리 형태로 정의 (이미 제공된 값 사용)
feature_importance = {
    'dow': 0.325531,
    'ssec': 0.238900,
    'nikkei': 0.195132,
    'brent': 0.076208,
    'kosdaq': 0.006516,
    'cny_krw': -0.000277,
    'usd_krw': -0.020725,
    'gold': -0.030892,
    'wti': -0.032822,
    'jpy_krw': -0.172221,
    'vix': -0.223214
}
# 피쳐 중요도 값을 배열 형태로 정의
feature_importance_arr = np.array(list(feature_importance.values()))

# 피쳐 가중치 적용
for feature, importance in feature_importance.items():
    if feature in df.columns:
        df[feature] *= importance

df.head()

,Unnamed: 0,날짜,kospi200,kosdaq,dow,nikkei,ssec,usd_krw,cny_krw,jpy_krw,콜금리,국고채,wti,brent,gold,vix,Target
0,1738,2017-04-17,278.10,4.101627,6717.957205,3581.698594,769.776413,-23.481839,-0.045603,-1.791736,1.23,2.165,-1.730704,4.218875,-39.909375,-3.272317,1
1,1737,2017-04-18,278.23,4.135901,6717.957205,3594.056304,763.694019,-23.481839,-0.045990,-1.814968,1.23,2.165,-1.730704,4.218875,-39.909375,-3.272317,0
2,1736,2017-04-19,276.49,4.144111,6680.963862,3596.712050,757.477841,-23.681214,-0.045996,-1.809285,1.23,2.165,-1.717575,4.183057,-39.977337,-3.218746,1
3,1735,2017-04-20,277.76,4.142873,6642.294034,3596.378375,757.814690,-23.702975,-0.045807,-1.792958,1.24,2.165,-1.661450,4.033689,-39.646793,-3.332585,1
4,1734,2017-04-21,280.05,4.137399,6699.008045,3633.504189,758.065535,-23.587537,-0.045641,-1.791236,1.24,2.165,-1.649962,4.038262,-39.659150,-3.158478,1


In [15]:
df=df.drop(['Unnamed: 0'], axis=1)
df.head()

,날짜,kospi200,kosdaq,dow,nikkei,ssec,usd_krw,cny_krw,jpy_krw,콜금리,국고채,wti,brent,gold,vix,Target
0,2017-04-17,278.10,4.101627,6717.957205,3581.698594,769.776413,-23.481839,-0.045603,-1.791736,1.23,2.165,-1.730704,4.218875,-39.909375,-3.272317,1
1,2017-04-18,278.23,4.135901,6717.957205,3594.056304,763.694019,-23.481839,-0.045990,-1.814968,1.23,2.165,-1.730704,4.218875,-39.909375,-3.272317,0
2,2017-04-19,276.49,4.144111,6680.963862,3596.712050,757.477841,-23.681214,-0.045996,-1.809285,1.23,2.165,-1.717575,4.183057,-39.977337,-3.218746,1
3,2017-04-20,277.76,4.142873,6642.294034,3596.378375,757.814690,-23.702975,-0.045807,-1.792958,1.24,2.165,-1.661450,4.033689,-39.646793,-3.332585,1
4,2017-04-21,280.05,4.137399,6699.008045,3633.504189,758.065535,-23.587537,-0.045641,-1.791236,1.24,2.165,-1.649962,4.038262,-39.659150,-3.158478,1


In [16]:
# 패키지 임포트
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score, roc_curve
from sklearn.preprocessing import MinMaxScaler

# 날짜 칼럼을 datetime 객체로 변환
df['날짜'] = pd.to_datetime(df['날짜'])
df.set_index('날짜', inplace=True)

# 타깃 변수 만들어주기
# 단순히 다음 날 오르면 1, 하락하면 0
df['Target'] = (df['kospi200'].shift(-1) > df['kospi200']).astype(int)


In [17]:
# 피처와 타겟을 분리합니다
features = df.drop('Target', axis=1)
target = df['Target']

# 피처 스케일링
scaler = MinMaxScaler()
scaled_features = scaler.fit_transform(features)

# LSTM 입력 형식으로 변환 (samples, time steps, features)
time_steps = 5  # 이전 5일의 데이터를 사용하여 예측

X = []
y = []

for i in range(time_steps, len(scaled_features)):
    X.append(scaled_features[i-time_steps:i])
    y.append(target.values[i])

X, y = np.array(X), np.array(y)


In [18]:
split_ratio = 0.7  # 훈련 세트 비율
split_index = int(split_ratio * len(X))

X_train, y_train = X[:split_index], y[:split_index]
X_test, y_test = X[split_index:], y[split_index:]

In [19]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

model = Sequential()
model.add(LSTM(units=128, return_sequences=True, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.1))
model.add(LSTM(units=64))
model.add(Dropout(0.1))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
earlystop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)


In [20]:
model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2, callbacks=[earlystop])

Epoch 1/100
31/31 [==============================] - 6s 49ms/step - loss: 0.6892 - accuracy: 0.5485 - val_loss: 0.7051 - val_accuracy: 0.4856
Epoch 2/100
31/31 [==============================] - 0s 15ms/step - loss: 0.6851 - accuracy: 0.5557 - val_loss: 0.6920 - val_accuracy: 0.5309
Epoch 3/100
31/31 [==============================] - 1s 24ms/step - loss: 0.6871 - accuracy: 0.5381 - val_loss: 0.7039 - val_accuracy: 0.4856
Epoch 4/100
31/31 [==============================] - 1s 25ms/step - loss: 0.6873 - accuracy: 0.5464 - val_loss: 0.6961 - val_accuracy: 0.4897
Epoch 5/100
31/31 [==============================] - 1s 28ms/step - loss: 0.6863 - accuracy: 0.5536 - val_loss: 0.6915 - val_accuracy: 0.5103
Epoch 6/100
31/31 [==============================] - 1s 25ms/step - loss: 0.6856 - accuracy: 0.5680 - val_loss: 0.6944 - val_accuracy: 0.4979
Epoch 7/100
31/31 [==============================] - 1s 22ms/step - loss: 0.6830 - accuracy: 0.5526 - val_loss: 0.6949 - val_accuracy: 0.5144
Epoch 

In [21]:
predictions = model.predict(X_test)
predictions = (predictions > 0.5).astype(int)  # 이진 분류 문제이므로 0.5를 기준으로 예측


17/17 [==============================] - 1s 6ms/step


In [22]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_test, predictions)
print(f"Accuracy: {accuracy}")


Accuracy: 0.508637236084453


-------

In [25]:
df = pd.read_csv('변화율merged_data.csv')
df.head()

,날짜,kospi200,kosdaq,dow,nikkei,ssec,usd_krw,cny_krw,jpy_krw,wti,brent,gold,vix
0,2024-05-14,371.04,0.91%,0.32%,0.35%,-0.06%,-0.44%,-0.11%,-0.28%,1.32%,0.69%,-0.99%,8.37%
1,2024-05-13,370.98,-1.13%,-0.21%,-0.20%,-0.24%,-0.44%,-0.52%,-0.73%,-0.20%,0.69%,-0.36%,8.37%
2,2024-05-10,371.08,-0.69%,0.32%,0.49%,0.01%,0.10%,0.31%,0.15%,0.47%,0.36%,0.78%,-2.38%
3,2024-05-09,368.84,-0.27%,0.85%,-0.35%,0.86%,0.50%,0.17%,0.17%,1.20%,0.71%,-0.08%,-1.74%
4,2024-05-08,374.09,0.14%,0.44%,-1.63%,-0.61%,0.20%,0.40%,0.03%,-0.73%,-0.73%,-0.30%,-1.93%


In [26]:
for column in df.columns:
    if column != '날짜':
        # 쉼표와 퍼센트 기호를 제거하고 숫자 형태로 변환
        df[column] = df[column].apply(lambda x: str(x).replace(',', '').replace('%', '') if isinstance(x, str) else x)
        # 문자열을 실수형으로 변환하고, 퍼센트 값을 실제 비율로 변환
        df[column] = df[column].astype(float) / 100

In [27]:
# 날짜 칼럼을 datetime 객체로 변환
df['날짜'] = pd.to_datetime(df['날짜'])
df.set_index('날짜', inplace=True)

# 타깃 변수 만들어주기
# 단순히 다음 날 오르면 1, 하락하면 0
df['Target'] = (df['kospi200'].shift(-1) > df['kospi200']).astype(int)

In [28]:
feature_importance = {
    'dow': 0.325531,
    'ssec': 0.238900,
    'nikkei': 0.195132,
    'brent': 0.076208,
    'kosdaq': 0.006516,
    'cny_krw': -0.000277,
    'usd_krw': -0.020725,
    'gold': -0.030892,
    'wti': -0.032822,
    'jpy_krw': -0.172221,
    'vix': -0.223214
}
# 피쳐 중요도 값을 배열 형태로 정의
feature_importance_arr = np.array(list(feature_importance.values()))

# 피쳐 가중치 적용
for feature, importance in feature_importance.items():
    if feature in df.columns:
        df[feature] *= importance

df.head()

,kospi200,kosdaq,dow,nikkei,ssec,usd_krw,cny_krw,jpy_krw,wti,brent,gold,vix,Target
날짜,,,,,,,,,,,,,
2024-05-14,3.7104,0.000059,0.001042,0.000683,-0.000143,0.000091,3.047000e-07,0.000482,-0.000433,0.000526,0.000306,-0.018683,0
2024-05-13,3.7098,-0.000074,-0.000684,-0.000390,-0.000573,0.000091,1.440400e-06,0.001257,0.000066,0.000526,0.000111,-0.018683,1
2024-05-10,3.7108,-0.000045,0.001042,0.000956,0.000024,-0.000021,-8.587000e-07,-0.000258,-0.000154,0.000274,-0.000241,0.005312,0
2024-05-09,3.6884,-0.000018,0.002767,-0.000683,0.002055,-0.000104,-4.709000e-07,-0.000293,-0.000394,0.000541,0.000025,0.003884,1
2024-05-08,3.7409,0.000009,0.001432,-0.003181,-0.001457,-0.000041,-1.108000e-06,-0.000052,0.000240,-0.000556,0.000093,0.004308,0


In [29]:
# 피처와 타겟을 분리합니다
features = df.drop('Target', axis=1)
target = df['Target']

# 피처 스케일링
scaler = MinMaxScaler()
scaled_features = scaler.fit_transform(features)

# LSTM 입력 형식으로 변환 (samples, time steps, features)
time_steps = 5  # 이전 5일의 데이터를 사용하여 예측

X = []
y = []

for i in range(time_steps, len(scaled_features)):
    X.append(scaled_features[i-time_steps:i])
    y.append(target.values[i])

X, y = np.array(X), np.array(y)

In [30]:
split_ratio = 0.7  # 훈련 세트 비율
split_index = int(split_ratio * len(X))

X_train, y_train = X[:split_index], y[:split_index]
X_test, y_test = X[split_index:], y[split_index:]

In [31]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
model = Sequential()
model.add(LSTM(units=128, return_sequences=True, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.1))
model.add(LSTM(units=64))
model.add(Dropout(0.1))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
earlystop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)



In [32]:
model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2, callbacks=[earlystop])

Epoch 1/100
31/31 [==============================] - 7s 50ms/step - loss: 0.6963 - accuracy: 0.4825 - val_loss: 0.6904 - val_accuracy: 0.5514
Epoch 2/100
31/31 [==============================] - 0s 14ms/step - loss: 0.6936 - accuracy: 0.5196 - val_loss: 0.6930 - val_accuracy: 0.5350
Epoch 3/100
31/31 [==============================] - 0s 15ms/step - loss: 0.6945 - accuracy: 0.4825 - val_loss: 0.6931 - val_accuracy: 0.5103
Epoch 4/100
31/31 [==============================] - 0s 15ms/step - loss: 0.6941 - accuracy: 0.5165 - val_loss: 0.6894 - val_accuracy: 0.5514
Epoch 5/100
31/31 [==============================] - 0s 15ms/step - loss: 0.6936 - accuracy: 0.4773 - val_loss: 0.6903 - val_accuracy: 0.5514
Epoch 6/100
31/31 [==============================] - 1s 18ms/step - loss: 0.6934 - accuracy: 0.5196 - val_loss: 0.6919 - val_accuracy: 0.5556
Epoch 7/100
31/31 [==============================] - 1s 22ms/step - loss: 0.6942 - accuracy: 0.5010 - val_loss: 0.6887 - val_accuracy: 0.5514
Epoch 

In [33]:
predictions = model.predict(X_test)
predictions = (predictions > 0.5).astype(int)  # 이진 분류 문제이므로 0.5를 기준으로 예측

17/17 [==============================] - 1s 6ms/step


In [34]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_test, predictions)
print(f"Accuracy: {accuracy}")

Accuracy: 0.5374280230326296
